In [1]:
import pandas as pd
import numpy as np
#from skimage.viewer import ImageViewer
from skimage.feature import canny
from skimage.measure import label
from imageviewing import series_to_array
from sklearn.ensemble import RandomForestClassifier
#from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.stats import ttest_ind
#import matplotlib.pyplot as plt
#from sklearn.cross_validation import train_test_split

/usr/local/lib/python2.7/dist-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


In [2]:
%matplotlib qt
#%matplotlib inline

In [3]:
with open('datapath.txt') as f:
    datapath=f.readlines()[0].rstrip()

In [4]:
train=pd.read_csv(datapath+'/train.csv', nrows=None, dtype=np.uint8)
pixels=train.columns.drop('label')


In [5]:
def bw_series_array(series,pixel=None):
    return series_to_array((series>0).astype(np.uint8),pixel_columns=pixel)*255

In [6]:
def bw_canny(series,pixel=None):
    bw_array=bw_series_array(series,pixel=pixel)
    return canny(bw_array)

In [7]:
def num_bw_edges(series,pixel=None):
    return label(bw_canny(series,pixel=pixel),return_num=True)[1]

In [8]:
%time bw_edges=train.apply(lambda x: num_bw_edges(x, pixel=pixels),axis=1)

CPU times: user 56.2 s, sys: 8.06 ms, total: 56.2 s
Wall time: 56.2 s


In [9]:
rfc=RandomForestClassifier(max_features=0.1,min_samples_split=6,min_weight_fraction_leaf=0.0,n_estimators=200)

In [13]:
bw_edges_error=cross_val_score(rfc,pd.concat([train[pixels],bw_edges],axis=1),train['label'],cv=5)

In [21]:
normal_error=pd.read_csv(datapath+'/simple_random_forest_ev_error.csv',header=None)[1].values

In [22]:
normal_error

array([ 0.96145152,  0.96382244,  0.9616621 ,  0.96343932,  0.96474512])

In [23]:
ttest_ind(normal_error,bw_edges_error)

Ttest_indResult(statistic=-2.7413352051864961, pvalue=0.025396759469517683)

In [27]:
label_edges=pd.concat([bw_edges,train[['label','pixel0']]],axis=1)
label_edges.columns=['edges','label','pixel0']

In [39]:
label_edges[['edges','label']].groupby(['label']).apply(lambda x: x.hist())

    

label
0    [[Axes(0.125,0.1;0.336957x0.8), Axes(0.563043,...
1    [[Axes(0.125,0.1;0.336957x0.8), Axes(0.563043,...
2    [[Axes(0.125,0.1;0.336957x0.8), Axes(0.563043,...
3    [[Axes(0.125,0.1;0.336957x0.8), Axes(0.563043,...
4    [[Axes(0.125,0.1;0.336957x0.8), Axes(0.563043,...
5    [[Axes(0.125,0.1;0.336957x0.8), Axes(0.563043,...
6    [[Axes(0.125,0.1;0.336957x0.8), Axes(0.563043,...
7    [[Axes(0.125,0.1;0.336957x0.8), Axes(0.563043,...
8    [[Axes(0.125,0.1;0.336957x0.8), Axes(0.563043,...
9    [[Axes(0.125,0.1;0.336957x0.8), Axes(0.563043,...
dtype: object

In [45]:
viewer=ImageViewer(bw_canny(train.loc[245]))
viewer.show()

[]

In [40]:
train[(train['label']==8)&(bw_edges==2)]

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
131,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
175,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
245,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
268,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
290,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
318,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
449,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
466,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We can see that this is far from a perfect predictor. It might need a little bit of work. There are a bunch of images with splotches off of the normal shape which we could probably remove. Also, it might be useful to throw out single pixel curves.

There are a bunch of images that under-estimate the number of edges. It's possible that converting to black and white causes some of those problems, but lots of people just fail to close their loops.

Let's see if this new feature helps though.

In [ ]:
#def remove_bw_sploches(series):
#    bw_array=bw_series_array(series)
#    labels, n = label(bw_array)

In [ ]:
train2=pd.read_csv(datapath+'/train.csv', nrows=100, dtype=np.uint8)

In [ ]:
train2[pixels]

In [ ]:
(train.loc[8][pixels]!=train2.loc[8][pixels]).sum()

In [ ]:
(np.reshape((train.loc[8][pixels]).values, (28,28))!=np.reshape((train2.loc[8][pixels]).values, (28,28))).sum()

In [ ]:
type(image2)

In [ ]:
image1=np.reshape((train.loc[8][pixels]).values, (28,28))
image2=np.reshape((train2.loc[8][pixels]).values, (28,28))
print((image1==image2).all())
viewer=ImageViewer(image2)
viewer.show()

In [ ]:
series_to_array(train.loc[76])

In [ ]:
viewer=ImageViewer(series_to_array(train2.loc[23]))
viewer.show()

In [ ]:
viewer=ImageViewer(series_to_array(train.loc[76],pixel_columns=pixels))
viewer.show()

In [ ]:
viewer=ImageViewer(canny(series_to_array(train.loc[75])))
viewer.show()

In [ ]:
viewer=view_image(train.loc[0][pixels])
viewer.show()


In [ ]:
train['bw_edges']=train.apply(num_bw_edges,axis=1)

In [ ]:
(train.loc[76]!=0).sum()

In [ ]:
train[train['bw_edges']==3]